In [123]:

import os
import xml.etree.ElementTree as ET
import re
import json

In [124]:
files = os.listdir('./txt_data/')

In [125]:
def parse_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    
    tokens_and_anns = []
    for children in root:
        for child in children:
            tokens_and_anns.append((child.text, child.attrib))
            
    return tokens_and_anns

In [126]:

def match_to_text(tokens_and_anns, filename):
    with open(filename) as f:
        text = f.read()
    text = text.replace('\\', '')
        
    running_idx = 0
    anns_to_keep = []    
    
    for token, ann in tokens_and_anns:
        # if token in '[]\{\}()?*':
        #     token = '\\'+token
        # print(token, ann)
        # search = re.search(token, text)
        match_indices = [(m.start(0), m.end(0)) for m in re.finditer(re.escape(token), text)]
        for indices in match_indices:
            if indices[0] >= running_idx:
                if ann['pi'] != 'O':
                    anns_to_keep.append([indices[0], indices[1], ann['pi'], token])
                running_idx = indices[1]
                break
        
    return anns_to_keep

In [127]:
def get_for_all(filename_list):
    filename_list = [file.removesuffix('.txt') for file in filename_list]
    all_anns = []
    
    for filename in filename_list:
        tokens_and_anns = parse_xml(os.path.join('./xml_data/', filename+'_export.xml'))
        anns_to_keep = match_to_text(tokens_and_anns, os.path.join('./txt_data/', filename+'.txt'))
        all_anns.append((filename, anns_to_keep))
        
    return all_anns

In [128]:
all_anns = get_for_all(files)

In [131]:
def create_jsons(all_anns):
    all_jsons = []
    for file, anns in all_anns:
        with open(os.path.join('./txt_data/', file+'.txt')) as f:
            text = f.read()
        json_to_be = {
            'id': file,
            'text': text.replace('\\', ''),
            'pair': "",
            'lang': "",
            'label': [ann[:3] for ann in anns],
            'comments': ["Generated using a BERT-based classifier."]
        }
        all_jsons.append(json_to_be)
        
        with open(os.path.join('./json_data/', file+'.json'), 'w', encoding='utf-8') as f:
            json.dump(json_to_be, f, ensure_ascii=False)
            
    with open(os.path.join('./json_data/', 'bert_classifier.jsonl'), 'w', encoding='utf-8') as f:
        for element in all_jsons:
            json.dump(element, f, ensure_ascii=False)
            f.write('\n')        
    
    
        
        

In [132]:
create_jsons(all_anns)